In [1]:
import pandas as pd

import sys
sys.path.append(r"C:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\myLibraries")
from MARScore.utils import *
from MARScore.score import MARSCore

from custom_score.utils import cleanString
from sklearn.cluster import SpectralClustering, Birch
from hdbscan import HDBSCAN

import matplotlib.pyplot as plt
import numpy as np

c:\Pro\Stages\A4 - DVRC\Work\Supervised-Learning-using-Unsupervised-Learning-Metrics-in-the-absence-of-Annotated-Data\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are

In [2]:
dataset = pd.read_json(r'C:\Pro\Stages\A4 - DVRC\Work\Datasets\pubmed\test.json', lines=True)
dataset = dataset[["article_text", "abstract_text"]]
cleaner = lambda x: ". ".join(x).replace("<S>", "").strip()
format_dot = lambda x: x.replace(" .", ".")
dataset.loc[:,"abstract_text"] = dataset["abstract_text"].replace(regex=r"\[[^\]]*\]", value="")
dataset.loc[:,"article_text"] = dataset["article_text"].replace(regex=r"\[[^\]]*\]", value="")
dataset.loc[:,"abstract_text"] = dataset["abstract_text"].map(cleaner)
dataset.loc[:,"article_text"] = dataset["article_text"].map(cleaner)
dataset.loc[:,"abstract_text"] = dataset["abstract_text"].map(cleanString)
dataset.loc[:,"article_text"] = dataset["article_text"].map(cleanString)
dataset.loc[:,"abstract_text"] = dataset["abstract_text"].map(format_dot)
dataset.loc[:,"article_text"] = dataset["article_text"].map(format_dot)
dataset = dataset.rename(columns={"abstract_text": "summary",
                        "article_text": "text"})

In [3]:
subset = dataset.iloc[:2, :]

### Corpus to Sentences

In [ ]:
import spacy

In [ ]:
def corpusToSentences(corpus):
    corpus_sentences = []
    nlp = spacy.load("en_core_web_sm")
    for indiv in corpus:
        doc = nlp(indiv)
        corpus_sentences.append([sent.text for sent in doc.sents])   
    return corpus_sentences

In [ ]:
res = corpusToSentences(subset["text"].tolist())

In [ ]:
print(res)

### Token to sentences

In [ ]:
def cleanMarkers(embs, labels, mode="all", ignore=["."]):
    """
    Removes vectors associated with noisy words such as stop words, punctuation, and BERT separator tokens.

    :param1 embs (list): List of words embeddings.
    :param2 labels (list): List of text token associated with each embedding.

    :output1 new_embs (list): Cleansed list of words embeddings.
    :output2 new_labels (list): Cleansed list of tokens.
    :output3 -1 (int): Error output.
    """
    token_indexes = [i for i in range(len(labels)) if (labels[i] != "[PAD]" and labels[i] != "[CLS]" and labels[i] != "[SEP]") or labels[i] in ignore]
    new_embs = [embs[i] for i in range(len(embs)) if i in token_indexes]
    new_labels = [labels[i] for i in range(len(labels)) if i in token_indexes]

    if mode == "all":
        return new_embs, new_labels
    elif mode == "emb":
        return new_embs
    elif mode == "lab":
        return new_labels
    else:
        return -1

In [ ]:
o, l = tokenizeCorpus(subset["text"][0])
v = vectorizeCorpus(o)

In [ ]:
v, l = cleanMarkers(v, l)

In [ ]:
print(" ".join(l))

In [ ]:
sentenced_tokens = corpusToSentences([" ".join(l)])
print(sentenced_tokens)

### Sentence Masks

In [ ]:
mask = [i for i, sent in enumerate(sentenced_tokens[0]) for _ in sent.split(" ")]

In [ ]:
mask

### Sentences clusters visualisation

In [ ]:
from matplotlib import colormaps

In [4]:
ms = MARSCore(subset["text"].to_list(),
              subset["summary"].to_list(),
              precision_level="s",
              ratio=5,
              n_allowed_elements=7,
              extraction_method="concat_l4",
              model=BertModel.from_pretrained('aitslab/biobert_huner_gene_v1'),
              tokenizer=BertTokenizer.from_pretrained('aitslab/biobert_huner_gene_v1'))
              #clusterizer=AffinityPropagation(random_state=0))
ms.compute()

In [5]:
def sentences_map(ms, index):
    sentences_mask = [i for i, sent in enumerate(ms.sentenced_labels[index]) for _ in sent.split(" ")]
    sentences_map = {0: []}
    for cluster_index, smask in zip(ms.clusters_labels[index], sentences_mask):
        if cluster_index in sentences_map.keys():
            sentences_map[cluster_index].append(smask)
        else:
            sentences_map[cluster_index] = [smask]
    inv_smap = {}
    for k,v in sentences_map.items():
        for x in v:
            inv_smap.setdefault(x,[]).append(k)
    return inv_smap

In [10]:
smap = sentences_map(ms, 0)

In [18]:
def selector(ms, s_map, index):
    selectedClusters = []
    for sel_index in ms.selectedIndexes[index]:
        selectedClusters.extend(smap[sel_index])
    return list(set(selectedClusters))

In [20]:
selectedClusters = selector(ms, smap, 0)

In [31]:
def score_viz(reductor, embs, labels, selected_clusters, embs_gold=None, labels_gold=None, labels_cluster=None, tf_values=None,):
    
    def colorize(label=None, glabels=[], clabel=None, cmap=[], mode="unclustered"):
        """
        Colorize vector's projections depending on the context. 

        :param1 label (string): Single Token.
        :param2 glabel (list): List of gold tokens.
        :param3 clabel (int): Cluster's index of the current token.
        :param4 cmap (color_map): Matplotlib color map.
        :param5 mode (string): Equals to <clustered>, <unclustered> to respectively colorize depending on gold's, cluster's belonging.

        :output color (string): CSS text color.  
        """
        comp_gold = True if label != None and glabels != [] else False
        assert label != None or glabels != [] or clabel != None, "ERROR: No labels detected"
        if mode == "unclustered":
            if comp_gold:
                color = "green" if label in glabels else 'red'
            else:
                color = "red"
        elif mode == "clustered":
            if clabel != None and cmap != []:
                color = cmap[clabel]
            else:
                color = "black"
        return color
    
    cmap = colormaps["viridis"].colors
    formated_embs = np.array(embs)
    formated_embs_gold = np.array(embs_gold)
    token_indexes = [i for i in range(len(labels)) if labels[i] != "[PAD]" and labels[i] != "[CLS]" and labels[i] != "[SEP]"]

    proj2D = np.transpose(embs)
    data = {"x": proj2D[0],
            "y": proj2D[1],
            "labels": labels,
            "clusters": labels_cluster}
    for k in data.keys():
        data[k] = [data[k][i] for i in range(len(data[k])) if i in token_indexes]

    token_indexes_gold = [i for i in range(len(labels_gold)) if labels_gold[i] != "[PAD]" and labels_gold[i] != "[CLS]" and labels_gold[i] != "[SEP]"]
    proj2D_gold = reductor.transform(formated_embs_gold).T
    data_gold = {"x": proj2D_gold[0],
                 "y": proj2D_gold[1],
                 "labels": labels_gold}
    for k in data_gold.keys():
        data_gold[k] = [data_gold[k][i] for i in range(len(data_gold[k])) if i in token_indexes_gold]

    traces = []
    for i in range(len(data['x'])):
        if labels_cluster[i] in selected_clusters:
            if data["clusters"] != None:
                color = colorize(clabel=data["clusters"][i], cmap=cmap, mode="clustered")
            else:
                color = colorize(labels=data['labels'][i], glabels=data_gold['labels'], mode="unclustered")
            trace = go.Scatter(
                x=[data['x'][i]],
                y=[data['y'][i]],
                mode='markers',
                marker=dict(size=9, color=color),
                line=dict(width=2, color="DarkSlateGrey"),
                text=["token: "+str(data['labels'][i]) +" || "+"tf   : "+str(tf_values[data['labels'][i]])+" || cluster: "+str(data["clusters"][i])],
                name=data['labels'][i]
            )
            traces.append(trace)
    
    for i in range(len(data_gold['x'])):
        trace = go.Scatter(
            x=[data_gold['x'][i]],
            y=[data_gold['y'][i]],
            mode='markers',
            marker=dict(size=9, color='red'),
            marker_symbol="diamond",
            line=dict(width=2, color="DarkSlateGrey"),
            text=["token: "+str(data_gold['labels'][i])],
            name=data_gold['labels'][i]
        )
        traces.append(trace) 
    layout = go.Layout(
    title='2D Scatter Plot',
    scene=dict(
        xaxis=dict(title='X'),
        yaxis=dict(title='Y')
    ),
    autosize=False,
    width=1000,
    height=1000
    )
    fig = go.Figure(data=traces, layout=layout)
    fig.update_yaxes(
        scaleanchor="x",
        scaleratio=1,
    )
    fig.show()

In [22]:
o_gold, l_gold = tokenizeCorpus(ms.gold[0])
v_gold = vectorizeCorpus(o_gold, method=ms.extraction_method)
v_gold, l_gold = cleanMarkers(v_gold, l_gold) 

In [26]:
print(selectedClusters)

[23, 25, 26, 28, 30, 32, 33, 34, 35, 41, 46, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 66, 67, 68, 69, 70, 71, 72, 73, 96, 97]


In [27]:
ms.visualize()

In [32]:
score_viz(ms.dim_reductor, ms.vectors[0], ms.labels[0], selectedClusters, v_gold, l_gold, ms.clusters_labels[0], ms.tokens_tfs[0])